# Initial setup

In [23]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame
from itertools import product

path = os.getcwd()
parent = os.path.dirname(path)
print(path)
print(parent)
#os.mkdir(path+'\outputs')


C:\Users\CMARCY\OneDrive - Environmental Protection Agency (EPA)\Desktop\py_projects\plantfile\NEEDS_project
C:\Users\CMARCY\OneDrive - Environmental Protection Agency (EPA)\Desktop\py_projects\plantfile


# Reading in NEEDS

In [24]:
#Reading in needs data
needs = pd.read_csv('outputs/needs_final.csv', low_memory=False)
needs['ID'] = needs['ORIS_Plant_Code'].astype(str) + '_' + needs['Unit_ID'].astype(str)
needs = needs.drop(columns='Unnamed: 0')
#print(needs.columns)
#needs = needs.drop(columns=['BART_Affected_Unit','Boiler/Generator/Committed_Unit', 'Bottom', 'C2G', 'C2G_Online_Year',
#        'CAMD_Database_UnitID','CCS','CCS_Removal','cap','Cogen?','Combustion_Turbine/IC_Engine','County','County_Code',
#        'DSI_Unit','FIPS5','Firing','FlueGasConditioning_Flag','HCL_Removal','Hg_EMF_for_BIT','Hg_EMF_for_LIG','Hg_EMF_for_SUB',
#        'Mercury_Controls','Mercury_Controls_Efficiency','Mode_1_NOx_Rate_(lbs/mmBtu)','Mode_2_NOx_Rate_(lbs/mmBtu)', 
#        'Mode_3_NOx_Rate_(lbs/mmBtu)','Mode_4_NOx_Rate_(lbs/mmBtu)','Modeled_Fuels','NOx_Comb_Control','NOx_Post-Comb_Control', 
#        'Commercial.Online.Year.Quarter','PM_Control','Plant_Name','Region_Name','SO2_Permit_Rate_(lbs/mmBtu)','Scrub_Match', 
#        'Scrubber_Efficiency','State_Code','State_Name','UniqueID','pca','resource_region','Retire_Age', 'NukeRetireBin','ctt', 
#        'Nuke80RetireYear','Nuke60RetireYear','NukeEarlyRetireYear','NukeRefRetireYear','IsExistUnit','coolingwatertech','ct',
#        'wst','Plant.NAICS.Description','RetireYearGiven','Retire_Yr_OG','RetireYear','Online_Yr_OG','ORIS_Plant_Code','Unit_ID'])
print(needs.columns)
#print(needs.dtypes)
#print(needs.head())


Index(['Plant_Name', 'UniqueID', 'ORIS_Plant_Code',
       'Boiler/Generator/Committed_Unit', 'Unit_ID', 'CAMD_Database_UnitID',
       'PlantType', 'Combustion_Turbine/IC_Engine', 'Region_Name',
       'State_Name', 'State_Code', 'County', 'County_Code', 'FIPS5', 'cap',
       'Fully.Loaded.Tested.Heat.Rate.Btu.kWh...Modeled',
       'Commercial.Online.Year', 'RetireYear', 'Firing', 'Bottom', 'Cogen?',
       'Modeled_Fuels', 'Wet/DryScrubber', 'Scrubber_Online_Year',
       'Scrubber_Efficiency', 'NOx_Comb_Control', 'NOx_Post-Comb_Control',
       'SCR_Online_Year', 'SNCR_Online_Year', 'PM_Control',
       'FlueGasConditioning_Flag', 'Mercury_Controls', 'ACI_Online_Year',
       'Mercury_Controls_Efficiency', 'SO2_Permit_Rate_(lbs/mmBtu)',
       'Mode_1_NOx_Rate_(lbs/mmBtu)', 'Mode_2_NOx_Rate_(lbs/mmBtu)',
       'Mode_3_NOx_Rate_(lbs/mmBtu)', 'Mode_4_NOx_Rate_(lbs/mmBtu)',
       'Hg_EMF_for_BIT', 'Hg_EMF_for_SUB', 'Hg_EMF_for_LIG', 'HCL_Removal',
       'DSI_Unit', 'DSI_Online_Yea

In [25]:
unique_scrub = pd.Series(needs['Wet/DryScrubber'].unique())
#print(unique_scrub)
is_wet = (needs['Wet/DryScrubber'] == 'Wet Scrubber') 
is_dry = (needs['Wet/DryScrubber'] == 'Dry Scrubber') 
#is_ri = (needs['Wet/DryScrubber'] == 'Reagent Injection') 
is_dsi = pd.notnull(needs['DSI_Online_Year'])
is_scr = pd.notnull(needs['SCR_Online_Year'])
is_sncr = pd.notnull(needs['SNCR_Online_Year'])
is_aci = pd.notnull(needs['ACI_Online_Year'])

needs.loc[is_wet,'SO2_Control'] = 'Wet FGD'
needs.loc[is_dry,'SO2_Control'] = 'Dry FGD'
needs.loc[is_dsi,'SO2_Control'] = 'DSI'
needs.loc[is_scr,'NOx_Control'] = 'SCR'
needs.loc[is_sncr,'NOx_Control'] = 'SNCR'
needs.loc[is_aci,'Hg_Control'] = 'ACI'

#needs.loc[is_wet,'WetScrub_Online_Year'] = needs['Scrubber_Online_Year']
#needs.loc[is_dry,'DryScrub_Online_Year'] = needs['Scrubber_Online_Year']
#needs.loc[is_ri,'RIScrub_Online_Year'] = needs['Scrubber_Online_Year']
#print(needs.columns)

#needs['Age'] = 2020 - needs['Commercial.Online.Year']
needs.loc[(2020 - needs['Commercial.Online.Year']< 40),'Age_of_Unit'] = '0 to 40 Years' 
needs.loc[(2020 - needs['Commercial.Online.Year']>= 40),'Age_of_Unit'] = '40 to 50 Years' 
needs.loc[(2020 - needs['Commercial.Online.Year']>= 50),'Age_of_Unit'] = 'Greater than 50 Years' 

print(needs.head())

      Plant_Name   UniqueID  ORIS_Plant_Code Boiler/Generator/Committed_Unit  \
0  Greene County     10_B_1               10                               B   
1  Greene County     10_B_2               10                               B   
2  Greene County  10_G_GT10               10                               G   
3  Greene County   10_G_GT2               10                               G   
4  Greene County   10_G_GT3               10                               G   

  Unit_ID  CAMD_Database_UnitID           PlantType  \
0       1                  19.0           O/G Steam   
1       2                  20.0           O/G Steam   
2    GT10                  21.0  Combustion Turbine   
3     GT2                  22.0  Combustion Turbine   
4     GT3                  23.0  Combustion Turbine   

  Combustion_Turbine/IC_Engine Region_Name State_Name  ...   ctt   wst  \
0                          NaN       S_SOU    Alabama  ...  none  none   
1                          NaN       S_S

# FOM and VOM data

In [26]:
fom = pd.read_csv('inputs/IPM_FOM.csv',encoding='latin1').replace('--', np.NaN)
fom.columns = fom.columns.str.replace(r'\s+', '_')
fom = fom.rename(columns={'Plant_Type':'PlantType','FOM_(2016$_/kW-Yr)':'FOM_2016'})

#print(fom.columns)

vom = pd.read_csv('inputs/IPM_VOM.csv',encoding='latin1').replace('--', np.NaN)
vom.columns = vom.columns.str.replace(r'\s+', '_')
vom = vom.rename(columns={'Capacity_Type':'PlantType','vom_updated':'VOM_2016'}).drop(columns='Variable_O&M_(2016$/mills/kWh)')
#print(vom.columns)

fom_nan = fom.isna().any(axis=1)
vom_nan = vom.isna().any(axis=1)
#print('fom total:',len(fom),', noctrl:',len(fom[fom_nan]),', ctrl:',len(fom[~fom_nan]))
#print('vom total:',len(vom),', noctrl:',len(vom[vom_nan]),', ctrl:',len(vom[~vom_nan]))
#print()

fom_noctrl = fom[fom_nan].drop(columns=['SO2_Control', 'NOx_Control', 'Hg_Control','Age_of_Unit_'])
vom_noctrl = vom[vom_nan].drop(columns=['SO2_Control', 'NOx_Control', 'Hg_Control'])
#print(fom_noctrl.columns)
#print(vom_noctrl.columns)

tech_lookup = pd.read_csv('inputs/tech_lookup.csv')
tech_added = pd.DataFrame([['Landfill Gas / Municipal Solid Waste', 'lfill-gas']], columns=('NEEDS_tech','ReEDS_tech'))
tech_lookup = tech_lookup.append(tech_added).rename(columns={'NEEDS_tech':'PlantType'})
#print(tech_lookup)
#print()

noctrl = pd.merge(fom_noctrl,vom_noctrl,on=['PlantType'],how='outer')
noctrl = pd.merge(noctrl,tech_lookup,on=['PlantType'],how='left').rename(columns={'FOM_2016':'FOM_nc','VOM_2016':'VOM_nc'})

Wind_row = noctrl[noctrl['PlantType'] == 'Wind']
Wind_FOM = Wind_row['FOM_nc'].item()
Wind_VOM = Wind_row['VOM_nc'].item()

MSW_row = noctrl[noctrl['PlantType'] == 'Landfill Gas / Municipal Solid Waste']
MSW_FOM = MSW_row['FOM_nc'].item()
MSW_VOM = MSW_row['VOM_nc'].item()

noctrl_added = pd.DataFrame([['Onshore Wind',Wind_FOM,Wind_VOM,'wind-ofs'],
                            ['Offshore Wind',Wind_FOM,Wind_VOM,'wind-ons'], 
                            ['Municipal Solid Waste',MSW_FOM,MSW_VOM,'lfill-gas'], 
                            ['Landfill Gas',MSW_FOM,MSW_VOM,'lfill-gas'], 
                            ['Energy Storage',29.94,0,'battery']],
                            columns=('PlantType','FOM_nc','VOM_nc','ReEDS_tech'))
noctrl = noctrl.append(noctrl_added).drop(columns=['ReEDS_tech'])
print(noctrl)


                              PlantType  FOM_nc  VOM_nc
0                               Biomass  134.52    7.29
1                             Fuel Cell    0.00   44.91
2                            Geothermal   93.51    5.49
3                                 Hydro   14.89    2.66
4                                  IGCC  102.34    3.16
5  Landfill Gas / Municipal Solid Waste  234.69    6.54
6                        Pumped Storage   17.27   10.17
7                              Solar PV   27.99    0.00
8                         Solar Thermal   77.93    0.00
9                                  Wind   28.18    0.00
0                          Onshore Wind   28.18    0.00
1                         Offshore Wind   28.18    0.00
2                 Municipal Solid Waste  234.69    6.54
3                          Landfill Gas  234.69    6.54
4                        Energy Storage   29.94    0.00


In [27]:
fom_ctrl = fom[~fom_nan].rename(columns={'Age_of_Unit_':'Age_of_Unit'})
vom_ctrl = vom[~vom_nan]
ctrl = pd.merge(fom_ctrl,vom_ctrl,on=['PlantType', 'SO2_Control', 'NOx_Control', 'Hg_Control'],how='outer')
ctrl = ctrl.replace({'No SO2 Control':np.NaN,'No NOx Control':np.NaN,'No Hg Control':np.NaN,'-':np.NaN})
#print(ctrl)
#print(len(ctrl))

unique_SO2 = pd.Series(ctrl['SO2_Control'].unique()).dropna()
unique_NOx = pd.Series(ctrl['NOx_Control'].unique()).dropna()
unique_Hg = pd.Series(ctrl['Hg_Control'].unique()).dropna()
unique_Age = pd.Series(ctrl['Age_of_Unit'].unique()).dropna()
#print(unique_SO2)
#print(unique_NOx)
#print(unique_Hg)
#print(unique_Age)

print(ctrl.head())


    PlantType SO2_Control NOx_Control Hg_Control            Age_of_Unit  \
0  Coal Steam         NaN         NaN        NaN          0 to 40 Years   
1  Coal Steam         NaN         NaN        NaN         40 to 50 Years   
2  Coal Steam         NaN         NaN        NaN  Greater than 50 Years   
3  Coal Steam         NaN         NaN        ACI          0 to 40 Years   
4  Coal Steam         NaN         NaN        ACI         40 to 50 Years   

   FOM_2016  VOM_2016  
0     28.34      1.43  
1     32.40      1.43  
2     41.63      1.43  
3     28.42      2.90  
4     32.49      2.90  


In [28]:
nuke = pd.read_csv('inputs/table_4-49_nuclear.csv')
nuke.columns = nuke.columns.str.replace(r'\s+', '_')
#print(nuke.columns)
nuke = nuke.rename(columns={'FOM_(2016$_/kW-yr)':'FOM_nuke','VOM_(2016_mills/kWh)':'VOM_nuke'})
#print(nuke.dtypes)
nuke['ID'] = nuke['ORIS_Code'].astype(str) + '_' + nuke['Unit_Id'].astype(str)

nuke = nuke.drop(columns=['ORIS_Code','Unit_Id','State', 'Plant_Name', 'On-Line_Year','Capacity_(MW)', 'Heat_Rate_(Btu/kWh)'])
#print(len(nuke))

avg_fom = nuke['FOM_nuke'].mean()
avg_vom = nuke['VOM_nuke'].mean()
#print(avg_fom)
#print(avg_vom)

nuke_added = pd.DataFrame([[avg_fom,avg_vom,'6020_1'],[avg_fom,avg_vom,'6149_1'],[avg_fom,avg_vom,'6040_1'],
                           [avg_fom,avg_vom,'1060_1'],[avg_fom,avg_vom,'2497_2'],[avg_fom,avg_vom,'8907_3']],
                          columns=('FOM_nuke','VOM_nuke','ID'))
nuke = nuke.append(nuke_added)
print(nuke.tail())
#print(len(nuke))


     FOM_nuke  VOM_nuke      ID
1  175.565726  0.143367  6149_1
2  175.565726  0.143367  6040_1
3  175.565726  0.143367  1060_1
4  175.565726  0.143367  2497_2
5  175.565726  0.143367  8907_3


# Merging DF

In [29]:
needs.loc[needs['PlantType'] == 'Combustion Turbine','Age_of_Unit'] = np.nan
needs.loc[needs['PlantType'] == 'Combined Cycle','Age_of_Unit'] = np.nan
needs.loc[needs['PlantType'] == 'O/G Steam','SO2_Control'] = np.nan
needs.loc[needs['PlantType'] == 'O/G Steam','Hg_Control'] = np.nan

needs_om = pd.merge(needs,noctrl,on=['PlantType'],how='left')
needs_om = pd.merge(needs_om,ctrl,on=['PlantType','SO2_Control', 'NOx_Control', 'Hg_Control', 'Age_of_Unit'],how='left')
needs_om = pd.merge(needs_om,nuke,on=['ID'],how='left')
#print(needs_om.columns)
#print()

om_cols = ['FOM_nc','VOM_nc','FOM_2016','VOM_2016','FOM_nuke','VOM_nuke']
needs_om['check'] = needs_om[om_cols].apply(lambda x:x.count(),axis=1)
om_err = needs_om['check'] != 2
#print(needs_om[om_err])
print('number of rows missing O&M data:',len(needs_om[om_err]))
print()

needs_om[om_cols]=needs_om[om_cols].fillna(0)
needs_om['FOM_2016'] = needs_om['FOM_2016'] + needs_om['FOM_nc'] + needs_om['FOM_nuke'] 
needs_om['VOM_2016'] = needs_om['VOM_2016'] + needs_om['VOM_nc'] + needs_om['VOM_nuke'] 
needs_om = needs_om.drop(columns=['FOM_nc','VOM_nc','FOM_nuke','VOM_nuke','check'])

needs_om[['WCOMB_V','WCOMB_F','WSNCR_V','WSNCR_F','WSCR_V','WSCR_F',
          'W_DSIV','W_DSIF','W_FFV','W_FFF','W_CAPAD']] = pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0]], index=needs_om.index)
#print(needs_om.head(2))

inflate = pd.read_csv('inputs/inflation_default.csv')
row_87 = inflate[inflate['t'] == 1987]
inf_87 = row_87['inflation_rate'].item()
row_16 = inflate[inflate['t'] == 2016]
inf_16 = row_16['inflation_rate'].item()
#print(inf_87)

needs_om['W_FOM'] = needs_om['FOM_2016'] * inf_87 / inf_16
needs_om['W_VOM'] = needs_om['VOM_2016'] * inf_87 / inf_16

#these techs don't have heat rates
no_hr = ['wind-ons','csp-ns','battery','hydro','DUPV','pumped-hydro','UPV','wind-ofs']
needs_om.loc[(needs_om['tech'].isin(no_hr)),'Fully.Loaded.Tested.Heat.Rate.Btu.kWh...Modeled'] = np.nan 
needs_om = needs_om.round(4)

needs_om.to_csv("outputs\\needs_final2.csv",index = False)
print(needs_om.head())


number of rows missing O&M data: 0

      Plant_Name   UniqueID  ORIS_Plant_Code Boiler/Generator/Committed_Unit  \
0  Greene County     10_B_1               10                               B   
1  Greene County     10_B_2               10                               B   
2  Greene County  10_G_GT10               10                               G   
3  Greene County   10_G_GT2               10                               G   
4  Greene County   10_G_GT3               10                               G   

  Unit_ID  CAMD_Database_UnitID           PlantType  \
0       1                  19.0           O/G Steam   
1       2                  20.0           O/G Steam   
2    GT10                  21.0  Combustion Turbine   
3     GT2                  22.0  Combustion Turbine   
4     GT3                  23.0  Combustion Turbine   

  Combustion_Turbine/IC_Engine Region_Name State_Name  ...  WSNCR_F WSCR_V  \
0                          NaN       S_SOU    Alabama  ...        0      0